In [ ]:
#hide   
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
from datetime import datetime as dt
from typing import *
import rfpye
from rfpye.parser import *
from rfpye.utils import *
from rfpye.blocks import *
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import *
import numpy as np
import pandas as pd

In [ ]:
#hide
%load_ext autoreload
%autoreload 2 

Caminho das pastas de entrada e saída

In [ ]:
entrada = Path(r'D:\OneDrive - ANATEL\Remote\rfeye002223')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

Função que mapeia os arquivos da pasta e retorna um lista generalizada `L`

In [ ]:
arquivos = get_files(entrada) ;  arquivos

(#1) [Path('D:/OneDrive - ANATEL/Remote/rfeye002223/rfeye002223_210519_T182401.bin')]

Peguemos 1 arquivo com somente 1 thread_id por bloco e 1 com múltiplos thread_ids por bloco.

In [ ]:
one_tid = arquivos[-1]
#mult_tid = arquivos[1]

In [ ]:
%%time
one = parse_bin(one_tid, btypes=[21,40,67])

Wall time: 3.06 s


In [ ]:
one

{'file_version': 23,
 'string': 'CRFS DATA FILE V023',
 'blocks': defaultdict(fastcore.foundation.L,
             {(21,
               0): (#1) [((36, 111), <rfpye.blocks.DType21 object at 0x0000020B1C68FA88>)],
              (40,
               1): (#9056) [((344, 399), <rfpye.blocks.DType40 object at 0x0000020B1C68FE48>),((15280, 15335), <rfpye.blocks.DType40 object at 0x0000020B1C998508>),((21000, 21055), <rfpye.blocks.DType40 object at 0x0000020B1C998748>),((25980, 26035), <rfpye.blocks.DType40 object at 0x0000020B1C998948>),((30960, 31015), <rfpye.blocks.DType40 object at 0x0000020B1C998C48>),((35940, 35995), <rfpye.blocks.DType40 object at 0x0000020B1C998FC8>),((111728, 111783), <rfpye.blocks.DType40 object at 0x0000020B1C9AEC08>),((116708, 116763), <rfpye.blocks.DType40 object at 0x0000020B1C250A08>),((121688, 121743), <rfpye.blocks.DType40 object at 0x0000020B1C9AEDC8>),((126668, 126723), <rfpye.blocks.DType40 object at 0x0000020B1C9B80C8>)...],
              (67,
             

On a normal conda environment the time of the following cell was
```python
%%timeit
multi = process_bin(mult_tid)
```
1.29 s ± 73.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%time
multi = parse_bin(mult_tid)

Wall time: 1.47 s


In [ ]:
multi.keys()

In [ ]:
multi['file_version']

In [ ]:
multi['string']

In [ ]:
multi['blocos']

In [ ]:
exemplo = multi['blocos'][(21,0)][0]

In [ ]:
bytes_map = exemplo[0]
bloco = exemplo[1]

In [ ]:
bin_data = bloco.data ; bin_data

In [ ]:
len(bin_data)

In [ ]:
getattrs(bloco)

### Mapeamento dos bytes do arquivo nos blocos

Vamos testar se o mapeamento do bloco nos bytes do arquivo está correto, para tal vamos ler o arquivo novamente.

In [ ]:
with open(mult_tid, mode='rb') as bfile:
    # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
    body = bfile.read()

Primeiramente criamos um Bloco Base - Cabeçalho, Dados e Rodapé com a função `create_base_block`.

In [ ]:
show_doc(create_base_block)

In [ ]:
block = create_base_block(body[slice(*bytes_map)])

Como vemos na documentação da função acima ela aceita arquivos abertos lidos como `bytes`. Passamos para ela o arquivo aberto `body`, mas somente selecionando deste os índices de bytes `bytes_map` retornados acima.
`body[slice(*bytes_map)]` é o mesmo que `body[bytes_map[0]:bytes_map[1]]` mas numa notação mais _pythônica_   

In [ ]:
block.data

A função de teste a seguir testa a igualdade do bloco de dados acima e do bloco de dados diretamente mapeado nos bins, o teste foi bem sucedido caso nada seja retornado

In [ ]:
test_eq(bin_data, block.data)

In [ ]:
%%time
metadados = extract_metadata(multi)

In [ ]:
metadados

In [ ]:
export_metadata(entrada, saida)